# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298668946116                   -0.85    5.4
  2   -8.300228040926       -2.81       -1.25    1.0
  3   -8.300449055566       -3.66       -1.89    3.5
  4   -8.300462399191       -4.87       -2.77    1.1
  5   -8.300464417188       -5.70       -3.10    4.6
  6   -8.300464530745       -6.94       -3.26    1.0
  7   -8.300464587547       -7.25       -3.40    9.1
  8   -8.300464617725       -7.52       -3.54    1.0
  9   -8.300464637095       -7.71       -3.73    1.0
 10   -8.300464641065       -8.40       -3.87    1.0
 11   -8.300464643709       -8.58       -4.17    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64142921158                   -0.70    5.8
  2   -16.67479606938       -1.48       -1.14    1.4
  3   -16.67922650279       -2.35       -1.88    2.2
  4   -16.67928163134       -4.26       -2.74    3.5
  5   -16.67928612647       -5.35       -3.12    5.5
  6   -16.67928621231       -7.07       -3.50    1.6
  7   -16.67928622198       -8.01       -3.99    1.4
  8   -16.67928622401       -8.69       -4.57    1.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30748952545                   -0.56    7.1
  2   -33.31495287898       -2.13       -1.00    1.1
  3   -33.33411596645       -1.72       -1.72    5.0
  4   -33.33429073909       -3.76       -2.61    2.6
  5   -33.33431269405       -4.66       -3.14    7.6
  6   -33.33694339156       -2.58       -2.56   11.1
  7   -33.33694338739   +   -8.38       -2.56    1.0
  8   -33.33694353753       -6.82       -2.75    1.0
  9   -33.33694384904       -6.51       -3.68    1.0
 10   -33.33694392404       -7.12       -4.22    1.0


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298347207381                   -0.85    5.1
  2   -8.300229485863       -2.73       -1.58    1.0
  3   -8.300315413062       -4.07       -2.15    5.5
  4   -8.300328418598       -4.89       -2.19    4.2
  5   -8.300464171529       -3.87       -3.59    1.2
  6   -8.300464586620       -6.38       -3.90    3.8
  7   -8.300464632521       -7.34       -4.22    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32707402836                   -0.56    7.5
  2   -33.27789084588   +   -1.31       -1.25    1.6
  3   +13.08261531157   +    1.67       -0.23   11.4
  4   -33.32546550806        1.67       -1.84    6.8
  5   -33.09929015993   +   -0.65       -1.25    4.9
  6   -32.26075412392   +   -0.08       -1.04    4.6
  7   -33.32349522431        0.03       -1.92    4.8
  8   -33.33080481316       -2.14       -2.03    2.9
  9   -33.33634911720       -2.26       -2.35    2.2
 10   -33.33683672794       -3.31       -2.86    2.2
 11   -33.33694026024       -3.98       -3.25    3.9
 12   -33.33694217239       -5.72       -3.35    2.5
 13   -33.33693624693   +   -5.23       -3.43    3.2
 14   -33.33694275779       -5.19       -3.87    2.4
 15   -33.33694382348       -5.97       -4.32    3.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.